In [10]:
import pandas as pd
import os

music_dir = os.path.abspath("mixed_up_data_talk/music")
noisy_dir = os.path.abspath("mixed_up_data_talk/noisy")

music_files = [f for f in os.listdir(music_dir)]
noisy_files = [f for f in os.listdir(noisy_dir)]

records = []
for f in music_files:
    records.append({f"filepath": os.path.join(music_dir,f), "label": "music"})
for f in noisy_files:
    records.append({"filepath": os.path.join(noisy_dir,f), "label": "noisy"})

df = pd.DataFrame.from_records(records)

df.to_csv("datasets/labels_mfcc_talk.csv", index=False)
print("Dataset salvato in datasets/labels_mfcc_talk.csv")

Dataset salvato in datasets/labels_mfcc_talk.csv


In [11]:
# Forza utilizzo della CPU per il train, più 'sbrigativo' che CUDA
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import pandas as pd
import numpy as np
import librosa
import tensorflow as tf

tf.config.optimizer.set_jit(False)

import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import joblib

CSV_PATH   = "datasets/labels_mfcc_talk.csv"
MODEL_OUT  = "yamnet_model_small_dataset.joblib"      # Salvo nella directory corrente, si può cambiare percorso
SR         = 16000                      


df = pd.read_csv(CSV_PATH)
label_map = {"music": 0, "noisy": 1}
df["label_num"] = df["label"].map(label_map)

file_label_list = list(zip(df["filepath"].tolist(),
                           df["label_num"].tolist()))

yamnet_model = hub.load("https://tfhub.dev/google/yamnet/1")

def extract_yamnet_embedding(wav_path, sr=16000):
    wav, _ = librosa.load(wav_path, sr=sr, mono=True)

    wav_tf = tf.convert_to_tensor(wav, dtype=tf.float32)

    scores, embeddings, spectrogram = yamnet_model(wav_tf)

    emb = tf.reduce_mean(embeddings, axis=0)
    return emb.numpy()


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

X = []
y = []
for path, label in file_label_list:
    emb = extract_yamnet_embedding(path)
    X.append(emb)
    y.append(label)
X = np.vstack(X)
y = np.array(y)

scaler = StandardScaler().fit(X)

norm = scaler.fit_transform(X)

lda = LinearDiscriminantAnalysis()
X_reduced = lda.fit_transform(norm,y)

X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y, test_size=0.2, random_state=42, stratify=y
)

In [13]:
clf = LogisticRegression(
    max_iter=500,
    class_weight='balanced',
    random_state=42
)
clf.fit(X_train, y_train)


print("Valutazione su Test Set:")
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['music','noisy']))
print("Matrice di Confusione:\n", confusion_matrix(y_test, y_pred))


print(f"Modello YAMNet salvato in {MODEL_OUT} …")
joblib.dump({'pca': lda, 'clf': clf}, MODEL_OUT)

def predict_noisy_probability(wav_path):

    emb = extract_yamnet_embedding(wav_path).reshape(1, -1)
    emb_scaled = scaler.transform(emb)         
    emb_lda    = lda.transform(emb_scaled)    
    prob_noisy = clf.predict_proba(emb_lda)[0, 1]
    return prob_noisy


Valutazione su Test Set:
              precision    recall  f1-score   support

       music       1.00      1.00      1.00        13
       noisy       1.00      1.00      1.00        71

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84

Matrice di Confusione:
 [[13  0]
 [ 0 71]]
Modello YAMNet salvato in yamnet_head.joblib …


In [14]:
if __name__ == "__main__":
    test_files = [
        "audio_test/music_pure.wav",
        "audio_test/noise_pure.wav",
        "audio_test/voice_base_music.wav",
        "audio_test/voice_base_pure.wav",
        "audio_test/voice_base_noise.wav"
        ]
    for test_file in test_files:
        if os.path.exists(test_file):
            prob = predict_noisy_probability(test_file)
            print(f"Probabilità che '{test_file}' sia noisy: {prob:.3f}")
        else:
            print(f"File di test non trovato: {test_file}")

Probabilità che 'audio_test/music_pure.wav' sia noisy: 0.000
Probabilità che 'audio_test/noise_pure.wav' sia noisy: 0.992
Probabilità che 'audio_test/voice_base_music.wav' sia noisy: 0.871
Probabilità che 'audio_test/voice_base_pure.wav' sia noisy: 0.995
Probabilità che 'audio_test/voice_base_noise.wav' sia noisy: 1.000


# Breve Descrizione
In questo caso, differentemente dal modello 'large_dataset', YAMNet è stato allenato su una mole di dati più piccola ma 'tascabile', nel senso che ci veniva più comodo mostrare i ragionamenti fatti non su .csv di diversi GB ma ridotte in dimensione, il che è sicuramente meglio in termini di consegna del progetto e discussione orale. Il modello ovviamente overfitta e restituisce accuracy folli ma semplicemente perché vede pochi dati, comunque funziona abbastanza bene classificando gli embeddings di YAMNet ma perché questi sono altamente informativi. Performance più veritiere si vedonon sul modello 'large_dataset'.